In [68]:
# from PIL import Image

# img = Image.open(r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\3.2-DataIngestion\page3_img1.png").convert("RGB")


In [ ]:
import os
from PIL import Image
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
def load_images(folder):
    images = []
    names = []

    for file in os.listdir(folder):
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            img = Image.open(os.path.join(folder, file)).convert("RGB")
            images.append(img)
            names.append(file)

    return images, names

model = SentenceTransformer("facebook/dino-vits16")



def image_to_vector(image):
    return model.encode(image)
images, names = load_images(r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\dev\Photos")

vectors = [image_to_vector(img) for img in images]

dimension = 512
index = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine


vectors_np = np.array(vectors).astype("float32")
index.add(vectors_np)

print("Total images stored:", index.ntotal)
faiss.write_index(index, "image_vectors.faiss")
import json

with open("metadata.json", "w") as f:
    json.dump(names, f)
query_img = Image.open(r"C:\Users\kumaw\Downloads\2025102916413218.jpg").convert("RGB")
query_vector = image_to_vector(query_img)

D, I = index.search(
    np.array([query_vector]).astype("float32"),
    k=5
)

print("Most similar images:")
for idx in I[0]:
    print(names[idx])




No sentence-transformers model found with name facebook/dino-vits16. Creating a new one with mean pooling.
d:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\gen\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kumaw\.cache\huggingface\hub\models--facebook--dino-vits16. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(m

KeyError: <class 'transformers.models.vit.configuration_vit.ViTConfig'>

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:

import shutil

SOURCE_DIR = r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\dev\Photos"
DEST_DIR = r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\dev\similar_results1111"

os.makedirs(DEST_DIR, exist_ok=True)

for idx in I[0]:
    src_path = os.path.join(SOURCE_DIR, names[idx])
    dst_path = os.path.join(DEST_DIR, names[idx])

    shutil.copy(src_path, dst_path)

print("✅ Similar images saved successfully!")


✅ Similar images saved successfully!


In [ ]:
import os
import json
import faiss
import numpy as np
from PIL import Image
from sentence_transformers import SentenceTransformer


# -----------------------------
# Load images
# -----------------------------
def load_images(folder):
    images = []
    names = []

    for file in os.listdir(folder):
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            img = Image.open(os.path.join(folder, file)).convert("RGB")
            images.append(img)
            names.append(file)

    return images, names


# -----------------------------
# Load MODEL (DINO v2)
# -----------------------------
model = SentenceTransformer("clip-ViT-L-14")


# -----------------------------
# Image → Vector (NORMALIZED)
# -----------------------------
def image_to_vector(image):
    vec = model.encode(image)
    vec = vec / np.linalg.norm(vec)   # 🔥 mandatory normalization
    return vec


# -----------------------------
# Load dataset images
# -----------------------------
images, names = load_images(
    r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\dev\Photos"
)

vectors = [image_to_vector(img) for img in images]

# -----------------------------
# FAISS index (CORRECT DIM)
# -----------------------------
dimension = 384  # 🔥 DINO output dimension
index = faiss.IndexFlatIP(dimension)

vectors_np = np.array(vectors).astype("float32")
index.add(vectors_np)

print("Total images stored:", index.ntotal)

faiss.write_index(index, "image_vectors.faiss")

with open("metadata.json", "w") as f:
    json.dump(names, f)


# -----------------------------
# QUERY IMAGE
# -----------------------------
query_img = Image.open(
    r"C:\Users\kumaw\Downloads\2025102916413218.jpg"
).convert("RGB")

query_vector = image_to_vector(query_img)

D, I = index.search(
    np.array([query_vector]).astype("float32"),
    k=5
)

print("\nMost similar images:")
for score, idx in zip(D[0], I[0]):
    print(names[idx], " | similarity:", round(float(score), 3))



d:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\gen\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kumaw\.cache\huggingface\hub\models--sentence-transformers--clip-ViT-L-14. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fal

In [4]:
from huggingface_hub import InferenceClient
import os
import json
import faiss
import numpy as np
from PIL import Image

# -----------------------------
# Load MODEL (Multimodal Embedding)
# -----------------------------
model = SentenceTransformer("clip-ViT-B-32")


# -----------------------------
# Load images
# -----------------------------
def load_images(folder):
    images = []
    names = []

    for file in os.listdir(folder):
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            img = Image.open(os.path.join(folder, file)).convert("RGB")
            images.append(img)
            names.append(file)

    return images, names


# -----------------------------
# Image → Vector (NORMALIZED)
# -----------------------------
def image_to_vector(image):
    # 🔥 InferenceClient does NOT support encode()
    vec = model.feature_extraction(image)

    vec = np.array(vec).flatten().astype("float32")
    vec = vec / np.linalg.norm(vec)  # mandatory normalization

    return vec


# -----------------------------
# Load dataset images
# -----------------------------
images, names = load_images(
    r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\dev\Photos"
)

vectors = [image_to_vector(img) for img in images]


# -----------------------------
# FAISS index
# -----------------------------
dimension = vectors[0].shape[0]  # 🔥 auto-detect (SAFE)
index = faiss.IndexFlatIP(dimension)

vectors_np = np.array(vectors).astype("float32")
index.add(vectors_np)

print("Total images stored:", index.ntotal)

faiss.write_index(index, "image_vectors.faiss")

with open("metadata.json", "w") as f:
    json.dump(names, f)


# -----------------------------
# QUERY IMAGE
# -----------------------------
query_img = Image.open(
    r"C:\Users\kumaw\Downloads\2025102916413218.jpg"
).convert("RGB")

query_vector = image_to_vector(query_img)

D, I = index.search(
    np.array([query_vector]).astype("float32"),
    k=5
)

print("\nMost similar images:")
for score, idx in zip(D[0], I[0]):
    print(names[idx], "| similarity:", round(float(score), 3))


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


AttributeError: 'SentenceTransformer' object has no attribute 'feature_extraction'

In [ ]:
import shutil

# -----------------------------
# AUTO SAVE SIMILAR IMAGES
# -----------------------------
SOURCE_DIR = r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\dev\Photos"
SAVE_DIR = r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\dev\similar_images"

os.makedirs(SAVE_DIR, exist_ok=True)

SIMILARITY_THRESHOLD = 0.75   # adjust if needed

saved_count = 0

for score, idx in zip(D[0], I[0]):
    if score >= SIMILARITY_THRESHOLD:
        src_path = os.path.join(SOURCE_DIR, names[idx])

        # save with similarity score in name
        dst_name = f"{round(float(score), 3)}_{names[idx]}"
        dst_path = os.path.join(SAVE_DIR, dst_name)

        shutil.copy(src_path, dst_path)
        saved_count += 1

print(f"\n✅ {saved_count} similar images saved in:\n{SAVE_DIR}")



In [ ]:
query_img = Image.open(r"D:\GenerativeAi\Gen ai\1-Basics+Of+Langchain\3.2-DataIngestion\page4_img2.png").convert("RGB")
query_vector = image_to_vector(query_img)

D, I = index.search(
    np.array([query_vector]).astype("float32"),
    k=5
)

print("Most similar images:")
for idx in I[0]:
    print(names[idx])


Most similar images:
DSC_2546 copy.jpg (1).jpeg
DSC_2546 copy.jpg.jpeg
DSC_8701.JPG.jpeg
DSC_5068.JPG.jpeg
DSC_5517.JPG.jpeg


In [ ]:
import os
import shutil

SOURCE_DIR = "images"
DEST_DIR = "similar_results"

os.makedirs(DEST_DIR, exist_ok=True)

for idx in I[0]:
    src_path = os.path.join(SOURCE_DIR, names[idx])
    dst_path = os.path.join(DEST_DIR, names[idx])

    shutil.copy(src_path, dst_path)

print("Similar images saved successfully!")


FileNotFoundError: [Errno 2] No such file or directory: 'images\\DSC_2546 copy.jpg (1).jpeg'